In [ ]:
!nvidia-smi

Sun Mar 20 10:34:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 11449, done.
remote: Total 11449 (delta 0), reused 0 (delta 0), pack-reused 11449
Receiving objects: 100% (11449/11449), 11.30 MiB | 14.11 MiB/s, done.
Resolving deltas: 100% (7902/7902), done.


In [ ]:
!pip install -qU wandb
!pip install -qU bbox-utility

     |████████████████████████████████| 1.7 MB 14.3 MB/s 
     |████████████████████████████████| 181 kB 71.9 MB/s 
     |████████████████████████████████| 144 kB 75.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 3.3 MB 14.9 MB/s 
     |████████████████████████████████| 58 kB 6.9 MB/s 
     |████████████████████████████████| 34.5 MB 1.3 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/input/happy-whale-and-dolphin.zip' -d '/content/data/'

In [ ]:
!rm -rf /content/data/labels
!rm -rf /content/data/my_fold_yolo5

In [ ]:
# !unzip '/content/drive/MyDrive/Colab Notebooks/whale_share/fold0_labels.zip' -d /content/data # origin
# !unzip '/content/drive/MyDrive/input/fold0_labels.zip' -d /content/data
# !unzip '/content/drive/MyDrive/Colab Notebooks/whale_share/fold1_labels.zip' -d /content/data
# !cp /content/data/labels/* /content/data
# !unzip '/content/drive/MyDrive/Colab Notebooks/whale_share/fold2_labels.zip' -d /content/data
# !cp /content/data/labels/* /content/data
# !unzip '/content/drive/MyDrive/Colab Notebooks/whale_share/fold3_labels.zip' -d /content/data
# !cp /content/data/my_fold_yolo5/* /content/data
!unzip '/content/drive/MyDrive/Colab Notebooks/whale_share/fold4_labels.zip' -d /content/data

In [ ]:
!rm -rf /content/data/*.txt

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob

import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')

from joblib import Parallel, delayed

from IPython.display import display, HTML

from matplotlib import animation, rc
rc('animation', html='jshtml')

In [ ]:
len(set(glob.glob('/content/data/*.txt')))

27857

In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    wandb.login(anonymous='must')
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

wandb: Currently logged in as: jcdata (use `wandb login --relogin` to force relogin)


To use your W&B account,
Go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. 
Get your W&B access token from here: https://wandb.ai/authorize


In [ ]:
FOLD      = 1 # which fold to train
DIM       = 1024*2
MODEL     = 'yolov5s6'
BATCH     = 8
EPOCHS    = 100

PROJECT   = 'whale' # w&b in yolov5
NAME      = f'{MODEL}-dim{DIM}-fold{FOLD}' # w&b for yolov5

REMOVE_NOBBOX = True # remove images with no bbox
ROOT_DIR  = '/content/data'
IMAGE_DIR = '/content/images' # directory to save images
LABEL_DIR = '/content/labels' # directory to save labels

In [ ]:
!rm -rf /content/images
!rm -rf /content/labels

In [ ]:
!mkdir -p {IMAGE_DIR}
!mkdir -p {LABEL_DIR}

In [ ]:
os.makedirs('/content/images/train', exist_ok=True)
os.makedirs('/content/images/val', exist_ok=True)
os.makedirs('/content/labels/train', exist_ok=True)
os.makedirs('/content/labels/val', exist_ok=True)

In [ ]:
for i in sorted(glob('/content/data/*.txt')):
    if 'README' in i:
        print(i)

/content/data/README.dataset.txt
/content/data/README.roboflow.txt


In [ ]:
from glob import glob
from shutil import copyfile

def make_dataset(mode='train'):
    txt_paths = sorted(glob('/content/data/*.txt'))
    if mode == 'train':
        pgb = txt_paths[:int(len(txt_paths)*0.8)]
    else:
        pgb = txt_paths[int(len(txt_paths)*0.8):]
    for i in pgb:
        if 'README' in i:
            continue
        file_name = i.split('/')[-1].split('.')[0].split('_')[0]
        copyfile('/content/data/test_images/'+file_name+'.jpg', f'/content/images/{mode}/' + file_name + '.jpg')
        f = open(i, 'r')
        k = open(f'/content/labels/{mode}/'+file_name+'.txt', 'w')
        for j in f.readlines():
            k.write(j)

        # img = plt.imread('/content/data/test_images/' + file_name+'.jpg')
        # plt.imshow(img)
        # plt.show()
        f.close()
        k.close()
        # break

In [ ]:
make_dataset(mode='train')
make_dataset(mode='val')

In [ ]:
import yaml

cwd = '/content/'

data = dict(
    path  = '/content',
    train =  '/content/images/train' ,
    val   =  '/content/images/val',
    nc    = 1,
    names = ['fin'],
    )

with open(os.path.join( cwd , 'gbr.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join( cwd , 'gbr.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
names:
- fin
nc: 1
path: /content
train: /content/images/train
val: /content/images/val



In [ ]:
%%writefile /content/hyp.yaml
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.1 # 0.30  # image rotation (+/- deg)
translate: 0.10  # image translation (+/- fraction)
scale: 0.10  # image scale (+/- gain)
shear: 0.0 # 2.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.0 # 0.2  # image mosaic (probability)
mixup: 0.0 # 0.5 # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)

Writing /content/hyp.yaml


In [ ]:
%cd /content
!rm -r /kaggle/working/yolov5
# !git clone https://github.com/ultralytics/yolov5 # clone
!cp -r /kaggle/input/yolov5-lib-ds /kaggle/working/yolov5
%cd yolov5
%pip install -qr requirements.txt  # install

from yolov5 import utils
display = utils.notebook_init()  # check

YOLOv5 🚀 v6.1-40-gb0ba101 torch 1.10.0+cu111 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)


Setup complete ✅ (8 CPUs, 51.0 GB RAM, 120.9/166.8 GB disk)


In [ ]:
!python train.py --img {DIM} \
--batch {BATCH} \
--epochs {EPOCHS} \
--data /content/gbr.yaml \
--hyp /content/hyp.yaml \
--weights {MODEL}.pt \
--project {PROJECT} --name {NAME} \
--exist-ok \
--single-cls